<a href="https://colab.research.google.com/github/vskChaithanya/Explanable_AI_LAB_ASS/blob/main/EX_AI_Lab01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#2303A52154 - Batch 45


In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression

data = {
    "Facebook_Ads": [1, 2, 3, 1, 2],
    "Bookings": [8, 15, 20, 10, 17]
}
df = pd.DataFrame(data)

X = df[["Facebook_Ads"]]
y = df["Bookings"]

model = LinearRegression()
model.fit(X, y)

slope = model.coef_[0]
intercept = model.intercept_
print(f"Regression Equation: y = {slope:.2f}x + {intercept:.2f}")

baseline = y.mean()
print(f"Baseline Bookings: {baseline:.2f}")

df["Predicted"] = model.predict(X)

df["SHAP_Value"] = df["Predicted"] - baseline

df["Final_Prediction"] = baseline + df["SHAP_Value"]

df["Error"] = df["Bookings"] - df["Predicted"]
df["Prediction_Type"] = df["Error"].apply(lambda e: "Underpredicted" if e > 0 else "Overpredicted")

print("\nFinal Table:")
print(df)

print("\nModel Summary:")
print(f"Slope (Effect of 1k₹ ad spend on bookings): {slope:.2f}")
print(f"Intercept (Bookings with 0 ad spend): {intercept:.2f}")
print("Higher Facebook ad spend increases predicted bookings linearly.")



Regression Equation: y = 5.71x + 3.71
Baseline Bookings: 14.00

Final Table:
   Facebook_Ads  Bookings  Predicted  SHAP_Value  Final_Prediction     Error  \
0             1         8   9.428571   -4.571429          9.428571 -1.428571   
1             2        15  15.142857    1.142857         15.142857 -0.142857   
2             3        20  20.857143    6.857143         20.857143 -0.857143   
3             1        10   9.428571   -4.571429          9.428571  0.571429   
4             2        17  15.142857    1.142857         15.142857  1.857143   

  Prediction_Type  
0   Overpredicted  
1   Overpredicted  
2   Overpredicted  
3  Underpredicted  
4  Underpredicted  

Model Summary:
Slope (Effect of 1k₹ ad spend on bookings): 5.71
Intercept (Bookings with 0 ad spend): 3.71
Higher Facebook ad spend increases predicted bookings linearly.


In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression

data = {
    "Ad_Clicks": [150, 100, 120, 90, 80],
    "Response_Time": [10, 15, 8, 12, 20],
    "Bookings": [300, 250, 280, 240, 200]
}
df = pd.DataFrame(data)

X = df[["Ad_Clicks", "Response_Time"]]
y = df["Bookings"]

model = LinearRegression()
model.fit(X, y)

coef_clicks, coef_response = model.coef_
intercept = model.intercept_
print(f"Regression Equation: y = {coef_clicks:.2f}*(Ad_Clicks) + {coef_response:.2f}*(Response_Time) + {intercept:.2f}")

baseline = y.mean()
print(f"Baseline Bookings: {baseline:.2f}")

df["Predicted"] = model.predict(X)

mean_clicks = df["Ad_Clicks"].mean()
mean_response = df["Response_Time"].mean()

df["SHAP_Ad_Clicks"] = coef_clicks * (df["Ad_Clicks"] - mean_clicks)
df["SHAP_Response_Time"] = coef_response * (df["Response_Time"] - mean_response)

df["Final_Prediction"] = baseline + df["SHAP_Ad_Clicks"] + df["SHAP_Response_Time"]

df["Error"] = df["Bookings"] - df["Predicted"]
df["Prediction_Type"] = df["Error"].apply(lambda e: "Underpredicted" if e > 0 else "Overpredicted")

print("\nFinal Table:")
print(df)

print("\nModel Summary:")
print(f"Effect of Ad Clicks: Each click changes bookings by {coef_clicks:.2f}")
print(f"Effect of Response Time: Each extra minute changes bookings by {coef_response:.2f}")
print("Positive SHAP values mean feature increased prediction above baseline, negative means it reduced prediction.")



Regression Equation: y = 0.89*(Ad_Clicks) + -3.44*(Response_Time) + 202.97
Baseline Bookings: 254.00

Final Table:
   Ad_Clicks  Response_Time  Bookings   Predicted  SHAP_Ad_Clicks  \
0        150             10       300  301.579280       37.250790   
1        100             15       250  240.018951       -7.095389   
2        120              8       280  281.857233       10.643083   
3         90             12       240  241.478206      -15.964624   
4         80             20       200  205.066330      -24.833860   

   SHAP_Response_Time  Final_Prediction     Error Prediction_Type  
0            10.32849        301.579280 -1.579280   Overpredicted  
1            -6.88566        240.018951  9.981049  Underpredicted  
2            17.21415        281.857233 -1.857233   Overpredicted  
3             3.44283        241.478206 -1.478206   Overpredicted  
4           -24.09981        205.066330 -5.066330   Overpredicted  

Model Summary:
Effect of Ad Clicks: Each click changes bookin

In [ ]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_diabetes
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

diabetes = load_diabetes()
X = pd.DataFrame(diabetes.data, columns=diabetes.feature_names)
y = pd.Series(diabetes.target, name="Disease_Progression")

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)

baseline = y_train.mean()
print(f"Diabetes Baseline Prediction: {baseline:.2f}")

preds = model.predict(X_test)

feature_means = X_train.mean()
shap_values = (X_test - feature_means) * model.coef_

final_preds = baseline + shap_values.sum(axis=1)

diabetes_results = X_test.copy()
diabetes_results["Actual"] = y_test.values
diabetes_results["Predicted"] = preds
diabetes_results["Final_Prediction"] = final_preds
diabetes_results["Error"] = diabetes_results["Actual"] - diabetes_results["Predicted"]

print("\nDiabetes Dataset SHAP Table (First 5 Rows):")
print(diabetes_results.head())

# part 2
student_data = pd.DataFrame({
    "Study_Time": [5, 3, 4, 2, 1],
    "Parental_Education": [3, 2, 4, 1, 2],
    "Absences": [2, 5, 1, 7, 10],
    "Final_Exam_Score": [90, 75, 88, 65, 50]
})

X_s = student_data.drop(columns=["Final_Exam_Score"])
y_s = student_data["Final_Exam_Score"]

model_s = LinearRegression()
model_s.fit(X_s, y_s)

baseline_s = y_s.mean()
print(f"\nStudent Exam Baseline Prediction: {baseline_s:.2f}")

preds_s = model_s.predict(X_s)

feature_means_s = X_s.mean()
shap_values_s = (X_s - feature_means_s) * model_s.coef_

final_preds_s = baseline_s + shap_values_s.sum(axis=1)

student_results = X_s.copy()
student_results["Actual"] = y_s
student_results["Predicted"] = preds_s
student_results["Final_Prediction"] = final_preds_s
student_results["Error"] = student_results["Actual"] - student_results["Predicted"]

print("\nStudent Dataset SHAP Table:")
print(student_results)



Diabetes Baseline Prediction: 153.74

Diabetes Dataset SHAP Table (First 5 Rows):
          age       sex       bmi        bp        s1        s2        s3  \
287  0.045341 -0.044642 -0.006206 -0.015999  0.125019  0.125198  0.019187   
211  0.092564 -0.044642  0.036907  0.021872 -0.024960 -0.016658  0.000779   
72   0.063504  0.050680 -0.004050 -0.012556  0.103003  0.048790  0.056003   
321  0.096197 -0.044642  0.051996  0.079265  0.054845  0.036577 -0.076536   
73   0.012648  0.050680 -0.020218 -0.002228  0.038334  0.053174 -0.006584   

           s4        s5        s6  Actual   Predicted  Final_Prediction  \
287  0.034309  0.032432 -0.005220   219.0  139.547558        139.547558   
211 -0.039493 -0.022517 -0.021788    70.0  179.517208        179.517208   
72  -0.002592  0.084492 -0.017646   202.0  134.038756        134.038756   
321  0.141322  0.098648  0.061054   230.0  291.417029        291.417029   
73   0.034309 -0.005142 -0.009362   111.0  123.789659        123.789659   

    

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

data = {
    "Study_Time": [5, 3, 4, 2, 1],
    "Parental_Education": [3, 2, 4, 1, 2],
    "Absences": [2, 5, 1, 7, 10],
    "Health": [4, 3, 5, 2, 1],
    "Final_Exam_Score": [90, 75, 88, 65, 50]
}
df = pd.DataFrame(data)

X = df.drop(columns=["Final_Exam_Score"])
y = df["Final_Exam_Score"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)

print("Regression Coefficients:")
for feature, coef in zip(X.columns, model.coef_):
    print(f"{feature}: {coef:.2f}")
print(f"Intercept: {model.intercept_:.2f}")

baseline = y_train.mean()
print(f"Baseline (Mean Final Exam Score): {baseline:.2f}")

y_pred = model.predict(X_test)

feature_means = X_train.mean()
shap_values = (X_test - feature_means) * model.coef_

final_predictions = baseline + shap_values.sum(axis=1)

results = X_test.copy()
results["Actual"] = y_test.values
results["Predicted"] = y_pred
results["Final_Prediction"] = final_predictions
for i, col in enumerate(X.columns):
    results[f"SHAP_{col}"] = shap_values.iloc[:, i]
results["Error"] = results["Actual"] - results["Predicted"]
results["Prediction_Type"] = results["Error"].apply(lambda e: "Underpredicted" if e > 0 else "Overpredicted")

print("\nFinal Results Table:")
print(results)



Regression Coefficients:
Study_Time: 3.89
Parental_Education: -1.43
Absences: -3.18
Health: 0.14
Intercept: 80.64
Baseline (Mean Final Exam Score): 73.25

Final Results Table:
   Study_Time  Parental_Education  Absences  Health  Actual  Predicted  \
1           3                   2         5       3      75  73.963768   

   Final_Prediction  SHAP_Study_Time  SHAP_Parental_Education  SHAP_Absences  \
1         73.963768              0.0                 0.713768           -0.0   

   SHAP_Health     Error Prediction_Type  
1          0.0  1.036232  Underpredicted  
